<a href="https://colab.research.google.com/github/Parkseojin2001/DeepLearningZeroToAll/blob/main/lab_04_1_multivariable_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multivariate Linear Regression

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

## Hypothesis Function: Naive

$$H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b$$

- 입력 변수 n개이면 weight(가중치)도 n개이다.

In [5]:
w1 = torch.zeros(1, requires_grad = True)
w2 = torch.zeros(1, requires_grad = True)
w3 = torch.zeros(1, requires_grad = True)
b =  torch.zeros(1, requires_grad = True)

In [6]:
# H(x) 계산
hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

## Hypothesis Function: Matrix

- matmul( )로 한 번에 계산
  
  a. 더 간결

  b. x의 길이가 바뀌어도 코드 변경 x

  c. 속도가 더 빠름

  $$ H(x) = Wx + b$$

In [9]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [8]:
W = torch.zeros((3, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [10]:
# H(x) 계산
hypothesis = x_train.matmul(W) + b

## Cost function: MSE

- Simple Linear Regression과 동일

$$cost(W) = \frac 1m \sum_{i = 1}^{m} (H(x^{(i)}) - y^{(i)})^2$$

In [11]:
cost = torch.mean((hypothesis - y_train) ** 2)

## Gradient Descent with torch.optim

$$\nabla W = \frac{\partial cost}{\partial W} = \frac 2m \sum_{i = 1}^{m} (Wx^{(i)} - y^{(i)})x^{(i)}$$

$$ W := W - \alpha \nabla W$$

In [12]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1e-5)

# optimizer 사용법
optimizer.zero_grad()
cost.backward()
optimizer.step()

## Full Code with torch.optim

In [13]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1e-5)

In [14]:
nb_epochs= 20
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  hypothesis = x_train.matmul(W) + b # or.mm or @

  # cost 계산
  cost = torch.mean((hypothesis - y_train) ** 2)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(epoch, nb_epochs,
                                                            hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371063
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

## nn.Module

- nn.Module을 상속해서 모델 생성

- nn.Linear(3, 1)

  - 입력 차원 : 3
  - 출력 차원 : 1

- Hypothesis 계산은 forward() 에서!

- Gradient 계산은 PyTorch가 알아서 : backward()

In [15]:
# 모델 초기화
W = torch.zeros((3, 1), requires_grad = True)
b= torch.zeros(1, requires_grad = True)

# H(x) 계산
hypothesis = x_train.matmul(W) + b # or .mm or @

In [23]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

model = MultivariateLinearRegressionModel()

## F.mse_loss

- torch.nn.functional에서 제공하는 loss function 사용

- 쉽게 다른 loss와 교체 가능!(l1_loss, smooth_l1_loss 등...)

In [17]:
# cost 계산
cost = torch.mean((hypothesis - y_train) ** 2)

In [24]:
prediction = model(x_train)
cost = F.mse_loss(prediction, y_train)

## Full Code with torch.optim

In [26]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
# W = torch.zeros((3, 1), requires_grad = True)
# b = torch.zeros(1, requires_grad = True)

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = optim.SGD([W, b], lr= 1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  hypothesis = model(x_train)

  # cost 계산
  cost = F.mse_loss(hypothesis, y_train)
  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{} hypothesis: {} Cost {:.6f}'.format(epoch, nb_epochs,
                                                           hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    1/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    2/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    3/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    4/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    5/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    6/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    7/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    8/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch    9/20 hypothesis: tensor([50.2635, 62.0156, 60.2688, 65.4965, 47.8591]) Cost 13140.943359
Epoch   10/20 hypoth